In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, IntegerType, ShortType
import datetime
import os

master = "spark://zy-ubuntu:7077"  
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--master {master} --driver-memory 4g --total-executor-cores 6 --executor-memory 8g --packages org.postgresql:postgresql:42.1.1 pyspark-shell'

In [2]:
spark = SparkSession.builder \
    .appName("title desc") \
    .getOrCreate()

In [3]:
title_basics_df = spark.read.csv("title.basics.tsv", sep=r'\t', header=True)
title_basics_df.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- isAdult: string (nullable = true)
 |-- startYear: string (nullable = true)
 |-- endYear: string (nullable = true)
 |-- runtimeMinutes: string (nullable = true)
 |-- genres: string (nullable = true)



In [4]:
title_basics_df = title_basics_df.withColumnRenamed('titleType', 'type') \
    .withColumnRenamed('primaryTitle', 'primary_title') \
    .withColumnRenamed('originalTitle', 'original_title') \
    .withColumnRenamed('isAdult', 'is_adult') \
    .withColumnRenamed('startYear', 'start_year') \
    .withColumnRenamed('endYear', 'end_year') \
    .withColumnRenamed('runtimeMinutes', 'runtime_minutes') \
    .drop('genres')
title_basics_df = title_basics_df.withColumn('end_year', F.when(F.col('end_year') == '\\N', F.lit(None)).otherwise(F.col('end_year'))) 
title_basics_df = title_basics_df.withColumn('runtime_minutes', F.when(F.col('runtime_minutes') == '\\N', F.lit(None)).otherwise(F.col('runtime_minutes'))) 
title_basics_df = title_basics_df.withColumn('is_adult', F.when(F.col('is_adult') == '0', F.lit(False)).otherwise(F.lit(True)))   

title_basics_df = title_basics_df.withColumn('start_year', F.col('start_year').cast(ShortType()))
title_basics_df = title_basics_df.withColumn('end_year', F.col('end_year').cast(ShortType()))
title_basics_df = title_basics_df.withColumn('runtime_minutes', F.col('runtime_minutes').cast(ShortType()))

title_basics_df.show()

+---------+-----+--------------------+--------------------+--------+----------+--------+---------------+
|   tconst| type|       primary_title|      original_title|is_adult|start_year|end_year|runtime_minutes|
+---------+-----+--------------------+--------------------+--------+----------+--------+---------------+
|tt0000001|short|          Carmencita|          Carmencita|   false|      1894|    null|              1|
|tt0000002|short|Le clown et ses c...|Le clown et ses c...|   false|      1892|    null|              5|
|tt0000003|short|      Pauvre Pierrot|      Pauvre Pierrot|   false|      1892|    null|              4|
|tt0000004|short|         Un bon bock|         Un bon bock|   false|      1892|    null|             12|
|tt0000005|short|    Blacksmith Scene|    Blacksmith Scene|   false|      1893|    null|              1|
|tt0000006|short|   Chinese Opium Den|   Chinese Opium Den|   false|      1894|    null|              1|
|tt0000007|short|Corbett and Court...|Corbett and Court

In [6]:
title_basics_df.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- type: string (nullable = true)
 |-- primary_title: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- is_adult: boolean (nullable = false)
 |-- start_year: string (nullable = true)
 |-- end_year: string (nullable = true)
 |-- runtime_minutes: string (nullable = true)



In [15]:
title_basics_df.select('type').distinct().show()

+------------+
|        type|
+------------+
|    tvSeries|
|tvMiniSeries|
|     tvMovie|
|   tvEpisode|
|       movie|
|   tvSpecial|
|       video|
|   videoGame|
|     tvShort|
|       short|
+------------+



In [5]:
# insert df into dim_casts table
title_basics_df.write.format('jdbc').options(
      url='jdbc:postgresql://localhost:5433/imdb',
      driver='org.postgresql.Driver',
      dbtable='dim_title_desc',
      user='admin',
      password='password'
      ).mode('append').save()

Py4JJavaError: An error occurred while calling o140.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 2.0 failed 4 times, most recent failure: Lost task 1.3 in stage 2.0 (TID 12) (192.168.0.188 executor 0): java.sql.BatchUpdateException: Batch entry 0 INSERT INTO dim_title_desc ("tconst","type","primary_title","original_title","is_adult","start_year","end_year","runtime_minutes") VALUES ('tt1110652','tvEpisode','Motherhood Issues','Motherhood Issues','FALSE','2007',NULL,NULL) was aborted: ERROR: column "start_year" is of type smallint but expression is of type character varying
  Hint: You will need to rewrite or cast the expression.
  Position: 155  Call getNextException to see other errors in the batch.
	at org.postgresql.jdbc.BatchResultHandler.handleError(BatchResultHandler.java:148)
	at org.postgresql.core.ResultHandlerDelegate.handleError(ResultHandlerDelegate.java:50)
	at org.postgresql.core.v3.QueryExecutorImpl.processResults(QueryExecut